In [1]:
import tensorflow as tf
import numpy as np

def train_model():
    inputs = tf.keras.Input(shape=(784,))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    X_train = np.random.rand(1000, 784).astype("float32")
    y_train = np.random.randint(0, 10, size=(1000,))

    model.fit(X_train, y_train, epochs=3, batch_size=32)
    model.save("simple_model.h5")
    return model

model = train_model()

Epoch 1/3
32/32 [==============================] - 1s 3ms/step - loss: 2.4068 - accuracy: 0.1000
Epoch 2/3
32/32 [==============================] - 0s 3ms/step - loss: 2.2960 - accuracy: 0.1460
Epoch 3/3
32/32 [==============================] - 0s 3ms/step - loss: 2.2383 - accuracy: 0.1550


d:\anaconda3\envs\tvm_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import tf2onnx
import tensorflow as tf

model = tf.keras.models.load_model("simple_model.h5")

input_signature = [tf.TensorSpec([None] + list(model.input_shape[1:]), dtype=tf.float32)]

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    output_path="model.onnx"
)

In [3]:
import tvm
import tvm.relay as relay
import onnx
import numpy as np
from tvm.contrib import graph_executor

onnx_model = onnx.load("model.onnx")

for input_tensor in onnx_model.graph.input:
    print(f"Input Name: {input_tensor.name}")
    print(f"Input Shape: {input_tensor.type.tensor_type.shape.dim}")

Input Name: args_0
Input Shape: [dim_param: "unk__6"
, dim_value: 784
]


In [4]:
mod, params = relay.frontend.from_onnx(onnx_model, shape={"args_0": (1, 784)})

In [5]:
target = "llvm"  

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

lib.export_library("model_tvm.dll")

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [6]:
dev = tvm.cpu()  
module = graph_executor.GraphModule(lib["default"](dev))

dummy_input = np.random.rand(1, 784).astype("float32")

module.set_input("args_0", dummy_input)  

module.run()

output = module.get_output(0).asnumpy()
print("TVM model output:", output)

TVM model output: [[0.11951701 0.07923162 0.13205251 0.0524486  0.08625779 0.11336085
  0.16997935 0.08701568 0.06630857 0.09382802]]
